<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>Cuts Optimization using Extra Gradient Boosting
<br></p><br>

Over the last years, **Machine Learning** tools have been successfully applied to problems in high-energy physics. For example, for the classification of physics objects. Supervised machine learning algorithms allow for significant improvements in classification problems by taking into account observable correlations and by learning the optimal selection from examples, e.g. from Monte Carlo simulations.


# Importing the Libraries

**Numpy** is a powerful library that makes working with python more efficient, so we will import it and use it as np in the code. **Pandas** is another useful library that is built on numpy and has two great objects *series* and *dataframework*. Pandas works great for *data ingestion* and also has *data visualization* features. From **Hipe4ml** we import **TreeHandler** and with the help of this function we will import our *Analysis Tree* to our notebook.

**Matplotlib** comes handy in plotting data while the machine learning is performed by **XGBOOST**. We will import data splitter from **Scikit-learn** as *train_test_split*. **Evaluation metrics** such as *confusion matrix*, *Receiver operating characteristic (ROC)*, and *Area Under the Receiver Operating Characteristic Curve (ROC AUC)*  will be used to asses our models.

A **Confusion Matrix** $C$ is such that $C_{ij}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$. Thus in binary classification, the count of true positives is $C_{00}$, false negatives $C_{01}$,false positives is $C_{10}$, and true neagtives is $C_{11}$.

If $ y^{'}_{i} $ is the predicted value of the $ i$-th sample and $y_{i}$ is the corresponding true value, then the fraction of correct predictions over $ n_{samples}$ is defined as 
$$
True \: positives (y,y^{'}) =  \sum_{i=1}^{n_{samples} } 1 (y^{'}_{i} = y_{i}=1)
$$ 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

import weakref 

from bayes_opt import BayesianOptimization
#from root_pandas import read_root


from data_cleaning import clean_df
from KFPF_lambda_cuts import KFPF_lambda_cuts
from plot_tools import AMS, preds_prob, plot_confusion_matrix, plt_sig_back
import tree_importer 
import uproot


#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 
        
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(8)

import gc

# Importing the data
CBM has a modified version of the cern's root software and it contains the simulated setup of CBM. Normally, a model generated input file, for example a URQMD 12 AGeV, is passed through different macros. These macros represent the CBM setup and it is like taking particles and passing them through a detector. These particles are registered as hits in the setup. Then particles' tracks are reconstructed from these hits using cellular automaton and Kalman Filter mathematics.


CBM uses the **tree** format of cern root to store information. To reduce the size of these root files a modified tree file was created by the name of Analysis tree. This Analysis tree file contains most of the information that we need for physics analysis. 

In this example, we download three Analysis Trees. The first one contains mostly background candidates for lambda i.e. protons and pions which do not come from a lambda. The second file contains mostly signal candidates of lamba i.e. it contains protons and pions which come from a lambda decay. The third one contains 10k events generated using URQMD generator with 12 AGeV energy.

In [4]:
(3/4)/(1-(3/4))

3.0

# Parallel processing

In [ ]:
a = ['index', 'LambdaCandidates_chi2geo', 'LambdaCandidates_chi2primneg',
       'LambdaCandidates_chi2primpos', 'LambdaCandidates_chi2topo',
       'LambdaCandidates_cosineneg', 'LambdaCandidates_cosinepos',
       'LambdaCandidates_cosinetopo', 'LambdaCandidates_distance',
       'LambdaCandidates_eta', 'LambdaCandidates_l', 'LambdaCandidates_ldl',
       'LambdaCandidates_mass', 'LambdaCandidates_p', 'LambdaCandidates_pT',
       'LambdaCandidates_phi', 'LambdaCandidates_rapidity','LambdaCandidates_issignal']
new_labels = []

for i in a:
    if 'LambdaCandidates_' in i:
        new_labels.append(i.replace('LambdaCandidates_',''))
    else:
        new_labels.append(i)

In [ ]:
df_clean_signal = uproot.open('dcm_5m_signal.root:plain_tree').arrays(library='pd')
df_clean_signal.columns = new_labels
gc.collect()

signal = df_clean_signal[ (df_clean_signal['mass']>df_clean_signal['mass'].mean()-3*df_clean_signal['mass'].std())
               & (df_clean_signal['mass']<df_clean_signal['mass'].mean()+3*df_clean_signal['mass'].std()) ]
#signal['issignal']=((signal['issignal']<2)*0 )
signal['issignal']=((signal['issignal']>0)*1)
signal["issignal"]=signal["issignal"].astype("int8")
#signal["issignal"].replace({1: 0, 2: 1}, inplace=True)
#del df_clean_signal

In [ ]:
df_clean_urqmd = uproot.open('urqmd_100k_uproot.root:t1').arrays(library='pd')
df_clean_urqmd.columns = new_labels
df_clean_urqmd['issignal']=((df_clean_urqmd['issignal']>0)*1)
df_clean_urqmd["issignal"]=df_clean_urqmd["issignal"].astype("int8")
#df_clean_urqmd = df_clean_urqmd[df_clean_urqmd['issignal']>0]
#df_clean_urqmd["issignal"].replace({1: 0, 2: 1}, inplace=True)

In [ ]:
df_clean =  uproot.open('dcm_100k_uproot.root:t1').arrays(library='pd')
df_clean.columns = new_labels
df_clean['issignal']=((df_clean['issignal']>0)*1)
df_clean["issignal"]=df_clean["issignal"].astype("int8")
#df_clean = df_clean[df_clean['issignal']>0]
#df_clean["issignal"].replace({1: 0, 2: 1}, inplace=True)

In [ ]:
def sig_bac_dist(df1):
    from matplotlib.backends.backend_pdf import PdfPages
    with PdfPages('multipage_pdf.pdf') as pdf:
        df = df1.copy()
        for i in df1.columns:
            list =['chi2geo', 'chi2primneg', 'chi2primpos','chi2topo', 'distance', 'ldl', 'rapidity','mass', 'pT']
            if i in list:
                df[i]=np.log(df[i])
            plt.figure(figsize=(8, 6))
            bin1 = 300 
            plt.hist(df[df['issignal']==0][i],bins = bin1, color = 'blue',alpha = 0.3,label='Background')
            plt.hist(df[df['issignal']==1][i],bins = bin1, color = 'red',label='Primaries', alpha =0.3)
            plt.hist(df[df['issignal']>1][i],bins = bin1, color = 'green',label='Secondaries', alpha =0.3)
            plt.yscale('log')
            #plt.grid()
            plt.ylabel('counts (log scale)', fontsize = 18)
            #plt.xlabel('$\chi^{2}_{geometrical}$', fontsize = 18)
            plt.legend(loc='upper right',fontsize=18)
            plt.tick_params(axis='both', which='major', labelsize=18)
            #ax.text(0, 1500, r'CBM Performance', fontsize=15)
            #ax.text(0, 500, r'URQMD, Au+Au @ 12 $A$GeV/$c$', fontsize=15)
            plt.title('URQMD, Au+Au @ 12 $A$GeV/$c$', fontsize=18)
            if i in list:
                plt.xlabel("log "+i, fontsize = 18)
            else:
                plt.xlabel(i, fontsize = 18)
            #plt.xlim([1.07,1.2])
            #plt.show()
            plt.tight_layout()
            pdf.savefig()
            plt.close()
        del df



## Renaming the columns

The above data frame object has some columns/features and for them at the very last column the true Monte Carlos information is available. This MC information tells us whether this reconstructed particle was originally produced as a decaying particle or not. So a value of 1 means that it is a true candidate and 0 means that it is not.

# Data Cleaning
Sometimes a data set contains entries which do not make sense. For example, infinite values or NaN entries. We clean the data by removing these entries. Ofcourse, we lose some data points but these outliers sometimes cause problems when we perform analysis. 

Since our experiment is a fixed target experiment so there are certain constraints which have to be applied on the data as well.

# Selecting Background and Signal
Our sample contains a lot of background (2178718) and somewhat signal candidates (36203). For analysis we will use a signal set of 4000 candidates and a background set of 12000 candidates. The background and signal candidates will be selected by using MC information.

In [ ]:
signal_selected= signal[(signal['mass']>1.1) & (signal['mass']<1.135)]
background_selected = df_clean_urqmd[(df_clean_urqmd['issignal'] == 0)
                & ((df_clean_urqmd['mass'] > 1.07)
                & (df_clean_urqmd['mass'] < 1.1) | (df_clean_urqmd['mass']>1.135) 
                   & (df_clean_urqmd['mass'] < 1.3))].sample(n=3*(signal_selected.shape[0]))
gc.collect()

#Let's combine signal and background
dfs = [signal_selected, background_selected]
df_scaled = pd.concat(dfs)

# Let's shuffle the rows randomly
df_scaled = df_scaled.sample(frac=1)
#del dfs, signal_selected, background_selected, signal
# Let's take a look at the top 10 entries of the df
df_scaled.iloc[0:10,:]
#del signal

In [ ]:
print(df_scaled.shape)
print(df_scaled[df_scaled['issignal']==1].shape)
print(df_scaled[df_scaled['issignal']==2].shape)

y = 0.5 * (np.log(E+P/E-P))


https://cbm-wiki.gsi.de/foswiki/bin/view/PWG/CbmCollisionEnergies


y = 0.5 * (np.log((12+10)/(12-10)))


using this the rapidity is 3.1992 for Ebeam =12.04 and pbeam =12 and mid rapidity is y/2=  1.5996

In [ ]:
fig, axs = plt_sig_back(df_scaled)
fig.set_figheight(5)
fig.set_figwidth(8)
axs.text(1.13, 6000, r'DCM-QGSM-SMM', color = 'magenta',  fontsize=15)
axs.text(1.13, 4000, r'Au+Au @ 12 $A$GeV/$c$', color = 'magenta',  fontsize=15)
axs.text(1.13, 2000, r'URQMD, Au+Au @ 12 $A$GeV/$c$', fontsize=15)
fig.savefig("hists.pdf")

# Creating Train and Test sets
To make machine learning algorithms more efficient on unseen data we divide our data into two sets. One set is for training the algorithm and the other is for testing the algorithm. If we don't do this then the algorithm can overfit and we will not capture the general trends in the data. 

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
cuts = [ 'chi2geo', 'chi2primneg', 'chi2primpos', 'distance', 'ldl']
#cuts = [ 'chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo',
#       'cosineneg', 'cosinepos', 'cosinetopo', 'distance', 'eta', 'l', 'ldl']


x = df_scaled[cuts].copy()

# The MC information is saved in this y variable
y =pd.DataFrame(df_scaled['issignal'], dtype='int8')

## Whole set

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
x_whole = df_clean[cuts].copy()
# The MC information is saved in this y variable
y_whole = pd.DataFrame(df_clean['issignal'], dtype='int8')

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>XGB Boost 
<br></p><br>

## Bayesian
In order to find the best parameters of XGB for our data we use Bayesian optimization. Grid search and and random search could also do the same job but bayesian is more time efficient. Stratify so that both train and test get the same ratio of signal to background.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=324, stratify=y)
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_whole, label = y_whole)
dtest1=xgb.DMatrix(x_test, label = y_test)
x_whole_1 = df_clean_urqmd[cuts].copy()
# The MC information is saved in this y variable
y_whole_1 = pd.DataFrame(df_clean_urqmd['issignal'], dtype='int8')
dtest2 = xgb.DMatrix(x_whole_1, label = y_whole_1)
gc.collect()

In [ ]:
import time
starttime = time.time()
def bo_tune_xgb(max_depth, gamma, alpha, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'alpha':alpha,
              'n_estimators': n_estimators,
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.3,
              'eval_metric': 'auc','tree_method':'hist','objective':'binary:logistic', 'nthread' : 7}
    cv_result = xgb.cv(params=params, dtrain=dtrain, num_boost_round=10, nfold=5)
    return  cv_result['test-auc-mean'].iloc[-1]

xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (4, 10),
                                             'gamma': (0, 1),
                                            'alpha': (2,20),
                                             'learning_rate':(0.01,1),
                                             'n_estimators':(100,500)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=5, init_points=5, acq='ei')
cpproot_time = time.time() - starttime
print(f"total time: {cpproot_time} sec")
#0.9983

### Hyper parameters

*subsample* [default=1]
Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.
range: (0,1]

*eta* [default=0.3, alias: learning_rate]
Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
range: [0,1]


*gamma* [default=0, alias: min_split_loss]
Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
range: [0,∞]


*alpha* [default=0, alias: reg_alpha]
L1 regularization term on weights. Increasing this value will make model more conservative.

*Lasso Regression* (Least Absolute Shrinkage and Selection Operator) adds “absolute value of magnitude” of coefficient as penalty term to the loss function.

# XGB models

In [ ]:
import time
starttime = time.time()

max_param = xgb_bo.max['params']
param= {'alpha': max_param['alpha'], 'gamma': max_param['gamma'], 'learning_rate': max_param['learning_rate'],
        'max_depth': int(round(max_param['max_depth'],0)), 'n_estimators': int(round(max_param['n_estimators'],0)), 
         'objective':'binary:logistic','tree_method':'hist','nthread' : 7}

#Fit/train on training data
bst = xgb.train(param, dtrain, num_boost_round=20)

#predicitions on training set
bst_train= pd.DataFrame(data=bst.predict(dtrain),  columns=["xgb_preds"])
y_train=y_train.set_index(np.arange(0,bst_train.shape[0]))
bst_train['issignal']=y_train['issignal']

#predictions on test set
bst_test = pd.DataFrame(data=bst.predict(dtest1),  columns=["xgb_preds"])
y_test=y_test.set_index(np.arange(0,bst_test.shape[0]))
bst_test['issignal']=y_test['issignal']

#ROC cures for the predictions on train and test sets
train_best, test_best = AMS(y_train, bst_train['xgb_preds'],y_test, bst_test['xgb_preds'])

#The first argument should be a data frame, the second a column in it, in the form 'preds'
preds_prob(bst_train,'xgb_preds', 'issignal',bst_test,'xgb_preds', 'issignal')

#Applying XGB on the 100k events data-set
df_clean['xgb_preds'] = bst.predict(dtest)
#preds_prob(df_clean,'xgb_preds', 'issignal','test')

df_clean_urqmd['xgb_preds'] = bst.predict(dtest2)
#del x_test, y_test, x_train, y_train, dtrain, dtest, x_whole, y_whole, x_whole_1, y_whole_1, dtest1, df_scaled
cpproot_time = time.time() - starttime
print(f"total time: {cpproot_time} sec")

In [ ]:
#The following graph will show us that which features are important for the model
ax = xgb.plot_importance(bst)
plt.rcParams['figure.figsize'] = [5, 3]
plt.show()
ax.figure.tight_layout() 
#ax.figure.savefig("hits.png")

In [ ]:
cut3 = 0.1
df3_base=df_clean_urqmd[(df_clean_urqmd['xgb_preds']>cut3) ]
#df3_base=df3_base[df3_base['xgb_preds1']>0.45]
#df3_base=df3_base[(df3_base['xgb_preds2']<0.45)&(df3_base['xgb_preds2']>0.15)]
fig, axs = plt.subplots(figsize=(12, 8))

range1= (1.105, 1.14)
bins1 = 150

#xgb

#issignal has 0,1,2 . So we convert all signals above zero to 1



df3_base['mass'].plot.hist(bins = bins1, range=range1, facecolor='red',alpha = 0.3,grid=True,sharey=True, label='XGB selected $\Lambda$s')
#df3_base[df3_base['issignal']==1]['mass'].plot.hist(bins = 300, range=range1,facecolor='blue',alpha = 0.3,grid=True,sharey=True, '\n True positives = \n (MC =1)\n signal in \n the distribution')
#df3_base[df3_base['issignal']==1]['mass'].plot.hist(bins = bins1, range=range1,facecolor='magenta',alpha = 0.3,grid=True,sharey=True )
df3_base[df3_base['issignal']==0]['mass'].plot.hist(bins = bins1, range=range1,facecolor='green',alpha = 0.3,grid=True,sharey=True, label ='\n False positives = \n (MC =0)\n background in \n the distribution')

plt.legend( fontsize = 18, loc='upper right')
#plt.rcParams["legend.loc"] = 'upper right'
plt.title("XGB selected $\Lambda$ candidates with a cut of %.3f "%cut3 +"on the XGB back probability distribution", fontsize = 18)
axs.set_xlabel("Mass (GeV/${c^2}$)", fontsize = 18)
plt.ylabel("Counts", fontsize = 18)
#axs.text(1.123, 4000, 'CBM Performance', fontsize=18)
#axs.text(1.123, 3500, 'URQMD, Au+Au @ 12A GeV/$c$', fontsize=18)
axs.tick_params(labelsize=18)
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

In [ ]:
print(df3_base[df3_base['issignal']==0].shape[0])
print(df3_base[df3_base['issignal']==1].shape[0])
print(df3_base[df3_base['issignal']>1].shape[0])

In [ ]:
def efficiency_plot_mass(df,signal_column, predictions_column, cut_value, range_min, range_max, bin1):
    from matplotlib import gridspec
    x_min, x_max = range_min , range_max
    range1= (x_min, x_max)

    fig, axs = plt.subplots(2, 1,figsize=(10,10), sharex=True, constrained_layout=True,  gridspec_kw={'width_ratios': [10],
                               'height_ratios': [8,4]})
    
    ns, bins, patches=axs[0].hist((df[(df[predictions_column]>cut_value) & (df[signal_column]==1)]['mass']),bins = bin1,histtype='step', range=range1,Fill=False, color='red', facecolor='red', linewidth=2)
    ns1, bins1, patches1=axs[0].hist((df[df[signal_column]==1]['mass']),bins = bin1,histtype='step', Fill=False, range=range1,facecolor='blue',linewidth=2)

    #plt.xlabel("Mass in GeV", fontsize = 15)
    axs[0].set_ylabel("log(counts)", fontsize = 18)
    axs[0].legend(('XGBoost TP','MC TP'), fontsize = 18, loc='upper right')
    axs[0].tick_params(axis='both', which='major', labelsize=18)
    axs[0].set_yscale('log')

    err = np.std(ns)
    err1 = np.std(ns1)
    corr_ns_ns1 = np.corrcoef(ns,ns1)[[0],[1]][0]
    err_dif = (ns / ns1) * (np.sqrt( ((err/ns)**2) + ((err1/ns1)**2)
                                      -2* ((corr_ns_ns1*err*err1)/(ns*ns1))))


    axs[1].hlines(y=1, xmin=x_min, xmax=x_max, colors='black', linestyles='dashed', label='')
    center = (bins[:-1] + bins[1:]) / 2
    axs[1].errorbar(center,  ns / ns1, yerr=err_dif,  fmt='o',
                     c='Blue', label='Background in predictions')


    plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 18)
    axs[1].set_ylabel("XGB / MC", fontsize = 18)
    axs[1].grid()
    axs[1].tick_params(axis='both', which='major', labelsize=18)
    fig.show()
    fig.tight_layout()
    return fig, axs

In [ ]:
efficiency_plot_mass(df_clean_urqmd,'issignal', 'xgb_preds', 0.96, 1.112, 1.12, 40)

## Confusion Matrix

By definition a confusion matrix $C$ is such that $C_{i, j}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$.

Thus in binary classification, the count of true positives is $C_{0,0}$, false positives is $C_{1,0}$, true negatives is $C_{1,1}$ and false negatives is $C_{0,1}$.

The following function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.

In [ ]:
import time
starttime = time.time()
cut3 = test_best
df_clean['xgb_preds'] = ((df_clean['xgb_preds']>cut3)*1)
cnf_matrix = confusion_matrix(df_clean['issignal'], df_clean['xgb_preds'], labels=[2,1,0])
#cnf_matrix = confusion_matrix(new_check_set['issignal'], new_check_set['new_signal'], labels=[1,0])
np.set_printoptions(precision=2)
fig, axs = plt.subplots(figsize=(10, 8))
axs.yaxis.set_label_coords(-0.04,.5)
axs.xaxis.set_label_coords(0.5,-.005)
plot_confusion_matrix(cnf_matrix, classes=['secondaries','primaries','background'], title='Confusion Matrix for XGB for cut > '+str(cut3))
plt.savefig('confusion_matrix_extreme_gradient_boosting_whole_data.png')
cpproot_time = time.time() - starttime
print(f"total time: {cpproot_time} sec")

# Cut visualization

In [ ]:
from matplotlib import gridspec

range1= (1.08, 1.15)


fig, axs = plt.subplots(2, 1,figsize=(10,10), sharex=True, constrained_layout=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})


ns, bins, patches=axs[0].hist((df3_base['mass']),bins = 50,histtype='step', range=range1,Fill=False, color='red', facecolor='red', linewidth=2)
ns1, bins1, patches1=axs[0].hist((new_check_set['mass']),bins = 50,histtype='step', Fill=False, range=range1,facecolor='blue',linewidth=2)
#plt.xlabel("Mass in GeV", fontsize = 15)
axs[0].set_ylabel("counts", fontsize = 18)
#axs[0].grid()
axs[0].legend(('XGBoost','KFPF '), fontsize = 18, loc='upper right')


axs[0].tick_params(axis='both', which='major', labelsize=18)
axs[0].set_yscale('log')

axs[1].hlines(y=1, xmin=1.112, xmax=1.12, colors='black', linestyles='dashed', label='')
center = (bins[:-1] + bins[1:]) / 2

err = np.std(ns)
err1 = np.std(ns1)
corr_ns_ns1 = np.corrcoef(ns,ns1)[[0],[1]][0]
err_dif = (ns / ns1) * (np.sqrt( ((err/ns)**2) + ((err1/ns1)**2)
                                      -2* ((corr_ns_ns1*err*err1)/(ns*ns1))))
axs[1].errorbar(center,  ns / ns1,   fmt='o',
                 c='Blue', label='Background in predictions')


    
    
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 18)
axs[1].set_ylabel("XGB / KFPF", fontsize = 18)
#axs[1].grid()
axs[1].tick_params(axis='both', which='major', labelsize=18)

fig.show()
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

# PyRoot

In [ ]:
#import sys, ROOT
from ROOT import TF1, TCanvas,TMath, TColor, TFile

import math
def truncate(number, decimals=2):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor


def background_selector(df):
    df1 = df[(df['mass']<1.108)]
    df2 = df[df['mass']>1.13]
    df3 = pd.concat([df1, df2])
    return df3['mass'] 

# Making Root trees

In [ ]:
import uproot
import awkward as ak
cut = 0.7
df3 = df_clean[df_clean['xgb_preds']>=cut]
df3 = df3[df3['issignal']>0]
df3 = df3[['pT', 'rapidity', 'mass','issignal','xgb_preds']]
df3.columns.values[[0,1,2,3,4]] = ['MCpT', 'MCrapidity','MCmass', 'MCissignal','MCxgb_preds']
df3["MCissignal"]=df3["MCissignal"].astype("float")
df3["MCxgb_preds"]=df3["MCxgb_preds"].astype("float")
df3_base = df_clean_urqmd[df_clean_urqmd['xgb_preds']>=cut]
df3_base3 = df3_base[['pT', 'rapidity', 'mass', 'issignal','xgb_preds']]
df3_base3["issignal"]=df3_base3["issignal"].astype("double")
df3_base3["xgb_preds"]=df3_base3["xgb_preds"].astype("double")

file = uproot.recreate("pt_y_yield_bdt_cut_0.8.root")
file["t1"] = df3_base3
file["t2"] = df3

# Efficiency 
Efficieny correction on just one configuration i.e lorenztian + 2nd order pol, 100 mass binings

In [ ]:
import numpy as np
import scipy.special as ss
ss.erf(3/np.sqrt(2))
#(2/np.pi)* np.arctan(7)
true_mc_in_recons =[]
pt_y_bin_for_yield_min=[]
pt_y_bin_for_yield_max=[]
y_bin_for_yield_max=[]
y_bin_for_yield_min=[]
df = df_clean[df_clean['xgb_preds']>=cut]

mass_range_min = [1.08]
fit_limit_low=[0,0.1* (df['mass'].describe()[2]),   0.2* (df['mass'].describe()[2]),
               1.23,
               df['mass'].describe()[1]+1.2*(df['mass'].describe()[2])+0.1* (df['mass'].describe()[2]),
                df['mass'].describe()[1]+1.2*(df['mass'].describe()[2])+0.2* (df['mass'].describe()[2])]


for mm in mass_range_min:
    for mmm in range(0,1,1):

        binning = [150]
        for b in binning:

            y_bin_low=-0.2
            y_bin_up =0
            for i in range(0,15,1):
                tot_sig_3_point_5_sigma, tot_sig_3_sigma, tot_sig_2_point_5_sigma, tot_sig_2_sigma = 0, 0, 0, 0
                tot_bac_3_sigma, tot_bac_3_point_5_sigma, tot_bac_2_point_5_sigma = 0, 0, 0
                
                y_bin_low = truncate(y_bin_low+0.2)
                y_bin_up = truncate(y_bin_up+0.2)
                df_y = df[(df['rapidity']>y_bin_low) & (df['rapidity']<y_bin_up)]
                pt_bin_low =-0.2
                pt_bin_up =0
                
                for i in range(0,15,1):
                    pt_bin_low = truncate(pt_bin_low+0.2)
                    #print(pt_bin_low)
                    pt_bin_up = truncate(pt_bin_up+0.2)
                    df_pt = df_y[(df_y['pT']>pt_bin_low) & (df_y['pT']<pt_bin_up)]
                    mc_counts = df_pt[df_pt['issignal']>0].shape[0]
                    #print(y_bin_low, y_bin_up, " pT ", pt_bin_low,pt_bin_up)
                    if df_pt.shape[0]>0:
                        true_mc_in_recons.append(mc_counts)
                        y_bin_for_yield_min.append(truncate(y_bin_low))
                        y_bin_for_yield_max.append(truncate(y_bin_up))
                        pt_y_bin_for_yield_min.append(pt_bin_low)
                        pt_y_bin_for_yield_max.append(pt_bin_up)
                    else:
                        y_bin_for_yield_min.append(truncate(y_bin_low))
                        y_bin_for_yield_max.append(truncate(y_bin_up))
                        pt_y_bin_for_yield_min.append(pt_bin_low)
                        pt_y_bin_for_yield_max.append(pt_bin_up)
                        true_mc_in_recons.append(mc_counts)

In [ ]:
import uproot
file =uproot.open("lambda_qa_dcm.root")
array1 = file["SimParticles_McLambda/SimParticles_rapidity_SimParticles_pT_McLambda;1"].to_numpy()
array2 = pd.DataFrame(data=array1[0])
array2.columns = np.arange(0,3,0.2)
array2

In [ ]:
#dcm
size = 15*15
pt_y_yields = pd.DataFrame(data=np.arange(0,size,1),columns = ['numbering'])
pt_y_yields['rapidity_min_MC'] = np.zeros(size)
pt_y_yields['pT_min_MC'] = np.zeros(size)

pt_y_yields['ratio_recons_sim']=np.zeros(size)
pt_y_yields['ratio_recons_mc']=np.zeros(size)
pt_y_yields['pT_min'] = pt_y_bin_for_yield_min
pt_y_yields ['pt_y_yields_MC']=np.zeros(size)
pt_y_yields['pt_y_yields_recons']=true_mc_in_recons
pt_y_yields['true_mc_in_recons'] = true_mc_in_recons
#pt_y_yields['total_mc_in_recons'] = dcm_clean_mc

for i in range(0,15):
    for j in range(0,15):
        pt_y_yields['rapidity_min_MC'].iloc[i+j*15]=0+j*0.2
    

for i in range(0,15):    
    pt_y_yields['pT_min_MC'].iloc[i]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+1*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+2*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+3*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+4*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+5*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+6*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+7*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+8*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+9*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+10*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+11*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+12*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+13*15]=i/5
    pt_y_yields['pT_min_MC'].iloc[i+14*15]=i/5
    


for i in range(0,15,1):
    pt_y_yields ['pt_y_yields_MC'].iloc[i]=array1[0][0][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+1*15]=array1[0][1][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+2*15]=array1[0][2][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+3*15]=array1[0][3][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+4*15]=array1[0][4][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+5*15]=array1[0][5][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+6*15]=array1[0][6][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+7*15]=array1[0][7][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+8*15]=array1[0][8][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+9*15]=array1[0][9][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+10*15]=array1[0][10][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+11*15]=array1[0][11][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+12*15]=array1[0][12][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+13*15]=array1[0][13][i]
    pt_y_yields['pt_y_yields_MC'].iloc[i+14*15]=array1[0][14][i]

for i in range(0,15*15,1):
#    pt_y_yields['ratio_recons_mc'].iloc[i]=dcm_clean_mc[i]/pt_y_yields['true_mc_in_recons'].iloc[i]
    pt_y_yields['ratio_recons_sim'].iloc[i]=true_mc_in_recons[i]/pt_y_yields['pt_y_yields_MC'].iloc[i]
#    pt_y_yields['pT_min'].iloc[i] = pt_y_bin_for_yield_min[i]
    #print("%.2f"%pt_y_yields['rapidity_min_MC'].iloc[i],"       ",pt_y_yields['pT_min_MC'].iloc[i],"    ", pt_y_yields['ratio'].iloc[i] )
#plt.plot(pt_y_yields['numbering'], pt_y_yields['ratio_recons_sim'], label='Reconstructed/Sim')
plt.scatter(pt_y_yields['numbering'], pt_y_yields['ratio_recons_sim'], label='Rencostructed/MC')
plt.legend()
plt.ylim([0.8,1.2])
plt.savefig("hists")
#pt_y_yields[(pt_y_yields['rapidity_min_MC']>1) & (pt_y_yields['rapidity_min_MC']<1.4) &(pt_y_yields['pT_min_MC']<1)&(pt_y_yields['pT_min_MC']>0)]
pt_y_yields[(pt_y_yields['numbering']>=100) & (pt_y_yields['numbering']<150)]

In [ ]:
file =TFile("lambda_qa_dcm.root")
#array1 = file["SimParticles_McLambda/SimParticles_rapidity_SimParticles_pT_McLambda;1"]
mc_spectra = file.Get("SimParticles_McLambda/SimParticles_rapidity_SimParticles_pT_McLambda;1");

In [ ]:
%jsroot off
import ROOT
from ROOT import TFile, TTree
from array import array
from ROOT import std

f = TFile('new_dcm_100_efficiency_pt_y_yield_bdt_cut_0.8.root','recreate')
t = TTree('t1','tree')

h4 = ROOT.TH2F("recons", "recons", 15,0,3,15,0,3);
h5 = ROOT.TH2F("Mc", "Mc", 15,0,3,15,0,3);
h6 = ROOT.TH2F("Mc in reconstructed", "Mc in reconstructed", 15,0,3,15,0,3);
h7 = ROOT.TH2F("Efficiency", "Efficiency", 15,0,3,15,0,3);
h8 = ROOT.TH2F("reconstructable_mc", "reconstructable_mc", 15,0,3,15,0,3);

bin1 = h4.FindBin(0);
bin2 = h4.FindBin(3);
for i in range(1,225):
    #recons.SetBinContent( (pt_y_yields1['rapidity_min'].iloc[i]), (pt_y_yields1['pT_min'].iloc[i]) ,pt_y_yields1['pt_y_yields'].iloc[i])
    y= (pt_y_yields['rapidity_min_MC'].iloc[i])
    pT=(pt_y_yields['pT_min_MC'].iloc[i])
    y_bin = int((y+0.1)/0.2 + 1);
    pT_bin = int((pT+0.1)/0.2 + 1);
    h4.SetBinContent(y_bin, pT_bin, pt_y_yields['pt_y_yields_recons'].iloc[i]);
    h5.SetBinContent(y_bin, pT_bin, pt_y_yields['pt_y_yields_MC'].iloc[i]);
    h6.SetBinContent(y_bin, pT_bin, pt_y_yields['true_mc_in_recons'].iloc[i]);
    h7.SetBinContent(y_bin, pT_bin, true_mc_in_recons[i]);
    #h8.SetBinContent(y_bin, pT_bin, dcm_clean_mc[i]);
    


#h4.Draw('colz')

#h5.Draw('colz')
#hist_2d.Draw('colz')
#ratio_recons_to_recons_mc=h4.Divide(h5)

#h6.Draw('colz')
ratio_recons_to_mc=h7.Divide(h5)

canvas = ROOT . TCanvas (" canvas ","", 950,800)
canvas.Draw()
canvas.SetGrid()
h7.Draw('colz')
#h7.GetZaxis().SetRangeUser(0,0.4)


f.Write()
f.Close()

In [ ]:
%jsroot on
from ROOT import TFile, TTree
from array import array
from ROOT import std


h4 = ROOT.TH2F("recons", "#frac{#Lambda_{recons}}{#epsilon_{DCM}}x#frac{1}{#Lambda_{sim}}", 15,0,3,15,0,3);
h5 = ROOT.TH2F("Mc", "Mc", 15,0,3,15,0,3);
h6 = ROOT.TH2F("Mc in reconstructed", "Mc in reconstructed", 15,0,3,15,0,3);
h7 = ROOT.TH2F("Efficiency", "Efficiency", 15,0,3,15,0,3);
h8 = ROOT.TH2F("reconstructable_mc", "reconstructable_mc", 15,0,3,15,0,3);
h9 = ROOT.TH2F("recons_new", "recons_new", 15,0,3,15,0,3);

bin1 = h4.FindBin(0);
bin2 = h4.FindBin(3);
for i in range(1,225):
    #recons.SetBinContent( (pt_y_yields1['rapidity_min'].iloc[i]), (pt_y_yields1['pT_min'].iloc[i]) ,pt_y_yields1['pt_y_yields'].iloc[i])
    y= (pt_y_yields['rapidity_min_MC'].iloc[i])
    pT=(pt_y_yields['pT_min_MC'].iloc[i])
    y_bin = int((y+0.1)/0.2 + 1);
    pT_bin = int((pT+0.1)/0.2 + 1);
    h4.SetBinContent(y_bin, pT_bin, true_mc_in_recons[i]);
    h5.SetBinContent(y_bin, pT_bin, pt_y_yields['pt_y_yields_MC'].iloc[i]);
    h6.SetBinContent(y_bin, pT_bin, pt_y_yields['true_mc_in_recons'].iloc[i]);
    h7.SetBinContent(y_bin, pT_bin, true_mc_in_recons[i]);
    #h8.SetBinContent(y_bin, pT_bin, dcm_clean_mc[i]);
    


#h4.Draw('colz')

#h5.Draw('colz')
#hist_2d.Draw('colz')
#ratio_recons_to_recons_mc=h4.Divide(h5)

#h4.Rebin(3)
#h7.Rebin(3)
#mc_spectra.Rebin(3)
#h6.Draw('colz')
ratio_recons_to_mc=h7.Divide(mc_spectra)
ratio1 = h4.Divide(h7)
ratio2 = h4.Divide(mc_spectra)

canvas = ROOT . TCanvas (" canvas ","", 950,800)
canvas.Draw()
canvas.SetGrid()
h4.Draw('colz')
h4.Draw("TEXT SAME");
h4.SetStats(0)
h4.GetZaxis().SetRangeUser(0.9,1.1)
canvas . Print ("/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/pT_rapidity_distribution_XGB_extracted_signal.png")

In [ ]:
def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.1) / multiplier

In [ ]:
for i in range(1,25):
    #recons.SetBinContent( (pt_y_yields1['rapidity_min'].iloc[i]), (pt_y_yields1['pT_min'].iloc[i]) ,pt_y_yields1['pt_y_yields'].iloc[i])
    y= (y_bin_for_yield_min[i])
    pT=(pt_y_bin_for_yield_min[i])
    y_bin = round(y*1.6)+1;
    pT_bin = round(pT*1.6);
    print(y_bin_for_yield_min[i],y_bin,pT_bin, true_mc_in_recons[i])

In [ ]:
1/0.6

In [ ]:
df_clean[(df_clean['pT']>2.6) & (df_clean['rapidity']>1.6) & (df_clean['rapidity']<1.8) & (df_clean['issignal']>0)]

In [ ]:
%jsroot on
from ROOT import TFile, TTree
from array import array
from ROOT import std


h4 = ROOT.TH2F("recons", "recons", 15,0,3,15,0,3);

bin1 = h4.FindBin(0);
bin2 = h4.FindBin(3);
for i in range(1,225):
    #recons.SetBinContent( (pt_y_yields1['rapidity_min'].iloc[i]), (pt_y_yields1['pT_min'].iloc[i]) ,pt_y_yields1['pt_y_yields'].iloc[i])
    y= (pt_y_yields['rapidity_min_MC'].iloc[i])
    pT=(pt_y_yields['pT_min_MC'].iloc[i])
    y_bin = int((y+0.1)/0.2 + 1);
    pT_bin = int((pT+0.1)/0.2 + 1);
    h4.SetBinContent(y_bin, pT_bin, true_mc_in_recons[i]);
    #h8.SetBinContent(y_bin, pT_bin, dcm_clean_mc[i]);
    
ratio1 = h4.Divide(h7)
ratio1 = h4.Divide(mc_spectra)
h4.GetZaxis().SetRangeUser(0.9,1.1)
canvas = ROOT . TCanvas (" canvas ","", 950,800)
canvas.Draw()
canvas.SetGrid()
h4.Draw('colz')

In [ ]:
f = TFile('new_urqmd_100_efficiency_pt_y_yield_bdt_cut_0.8.root','recreate')
t = TTree('t1','tree')

h4 = ROOT.TH2F("recons_urqmd", "recons_urqmd", 15,0,3,15,0,3);
h5 = ROOT.TH2F("Mc_urqmd", "Mc_urqmd", 15,0,3,15,0,3);

bin1 = h4.FindBin(0);
bin2 = h4.FindBin(3);
for i in range(1,225):
    #recons.SetBinContent( (pt_y_yields1['rapidity_min'].iloc[i]), (pt_y_yields1['pT_min'].iloc[i]) ,pt_y_yields1['pt_y_yields'].iloc[i])
    y= (pt_y_yields['rapidity_min_MC'].iloc[i])
    pT=(pt_y_yields['pT_min_MC'].iloc[i])
    y_bin = int((y+0.1)/0.2 + 1);
    pT_bin = int((pT+0.1)/0.2 + 1);
    h5.SetBinContent(y_bin, pT_bin, pt_y_yields['pt_y_yields_MC'].iloc[i]);
    h4.SetBinContent(y_bin, pT_bin, true_mc_in_recons[i]);
    #h8.SetBinContent(y_bin, pT_bin, dcm_clean_mc[i]);
    


#h4.Draw('colz')

#h5.Draw('colz')
#hist_2d.Draw('colz')
#ratio_recons_to_recons_mc=h4.Divide(h5)


#h7.GetZaxis().SetRangeUser(0,0.4)


f.Write()
f.Close()

In [ ]:
def sigma(df:pd.DataFrame, pid):
    mean = df[df['pid']==pid]['mass2'].mean()
    std = df[df['pid']==pid]['mass2'].std()
    in_sigma = ([df['pid']==pid])&(df['mass2'] > (mean-std)) & (df['mass2'] < (mean+std))
    return df[in_sigma]